<a href="https://colab.research.google.com/github/RishabhRanjan1425/letsupgrade1/blob/main/Rishabhproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [12]:
# loading Data
dataset = (r'C:\Users\DELL\Desktop\datasets\Capstone_project')

In [13]:
# setting image size and batch size
img_size = (180, 180)
batch_size = 32

In [14]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
# Load the list of image files
covid_images = [os.path.join(dataset, 'COVID', img) for img in os.listdir(os.path.join(dataset, 'COVID'))]
non_covid_images = [os.path.join(dataset, 'non-COVID', img) for img in os.listdir(os.path.join(dataset, 'non-COVID'))]

In [ ]:
# Split the data into train and validation sets
covid_train, covid_val = train_test_split(covid_images, test_size=0.2, random_state=42)
non_covid_train, non_covid_val = train_test_split(non_covid_images, test_size=0.2, random_state=42)

In [ ]:
train_df = pd.DataFrame({'filename': covid_train + non_covid_train, 'class': ['covid']*len(covid_train) + ['non-covid']*len(non_covid_train)})
val_df = pd.DataFrame({'filename': covid_val + non_covid_val, 'class': ['covid']*len(covid_val) + ['non-covid']*len(non_covid_val)})

Now we will be resizing the image to same size

In [ ]:
# Create train and validation generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=img_size,#here in the place of target_size we are resizing the images that can fix the all image sizes in to same sizes
    batch_size=batch_size,
    class_mode='binary',

)

validation_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=img_size,#here in the place of target_size we are resizing the images that can fix the all image sizes in to same sizes
    batch_size=batch_size,
    class_mode='binary',

)

In [ ]:
# importing librarys which is used to build architeture
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models,callbacks

In [ ]:
res50 = ResNet50(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)

In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization

In [ ]:
# Fully Connected Layers

flatten = Flatten()(res50.output)

dense = Dense(512, activation = 'relu')(flatten)
dense = Dropout(0.5)(dense)
dense = Dense(128, activation = 'relu')(dense)
dense = Dropout(0.3)(dense)

# Output Layer
prediction = Dense(1, activation = 'sigmoid')(dense)

In [ ]:
# Fully Connected Layers

flatten = Flatten()(res50.output)

dense = Dense(512, activation = 'relu')(flatten)
dense = Dropout(0.5)(dense)
dense = Dense(128, activation = 'relu')(dense)
dense = Dropout(0.3)(dense)

# Output Layer
prediction = Dense(1, activation = 'sigmoid')(dense)

In [ ]:
model1 = Model(inputs = res50.input, outputs = prediction)

In [ ]:
model1.summary()#summary of model

In [ ]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

In [ ]:
history1 = model1.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=[early_stopping],

)

In [ ]:
test_results = model1.evaluate(validation_generator)
print(f"Test Loss: {test_results[0]}, Test Accuracy: {test_results[1]}")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

y_true = validation_generator.classes  # True labels
y_pred_prob = model1.predict(validation_generator)  # Predicted probabilities

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)  # Adjust threshold as needed

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)


print("Validation Accuracy:", accuracy)
print("Validation Precision:", precision)
print("Validation Recall:", recall)
print("Validation F1 Score:", f1)

print("\nConfusion Matrix:")
print(conf_matrix)

In [ ]:
import cv2
import matplotlib.pyplot as plt
def cov():

    input_image_path =input()
    input_image = cv2.imread(input_image_path)

    plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    # cv2.imshow("Image",input_image)
    input_image_resized = cv2.resize(input_image, (180,180))

    input_image_scaled = input_image_resized/255

    input_image_reshaped = np.reshape(input_image_scaled, [1,180,180,3])

    input_prediction = model1.predict(input_image_reshaped)

    print(input_prediction)
    if input_prediction>0.5:
        print("Non-covid")
    else:
        print("covid")

In [ ]:
#calling a function
cov()

In [ ]:
#calling a function
cov()

In [ ]:
model1.save("Ct_Scan_Covid.h5")

In [ ]:

# library to load the model
from tensorflow.keras.models import load_model

In [ ]:
s_model = load_model('/kaggle/working/Ct_Scan_Covid.h5')

In [ ]:
import cv2
import matplotlib.pyplot as plt
def saved_model():

    input_image_path =input()
    input_image = cv2.imread(input_image_path)

    plt.imshow(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    # cv2.imshow("Image",input_image)
    input_image_resized = cv2.resize(input_image, (180,180))

    input_image_scaled = input_image_resized/255

    input_image_reshaped = np.reshape(input_image_scaled, [1,180,180,3])

    input_prediction = s_model.predict(input_image_reshaped)

    print(input_prediction)
    if input_prediction>0.5:
        print("Non-covid")
    else:
        print("covid")

In [ ]:
saved_model()